In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def validar_assinatura(caminho_arquivo):

    options = webdriver.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--start-maximized') 
    options.add_argument('--headless') 
    
    driver = webdriver.Chrome(options=options)
    
    try:
        driver.get('https://validar.iti.gov.br')

        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Aceitar")]'))
            ).click()
        except:
            print("Nenhum banner de cookies detectado.")
        
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="file"]'))
        ).send_keys(caminho_arquivo)

        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'acceptTerms'))
        ).click()
        
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(., "Validar")]'))
        ).click()

        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, 'swal2-html-container'))
            )

            exception = driver.find_element(By.ID, 'swal2-html-container').get_property("innerText")

            return exception
        except:
            WebDriverWait(driver, 30).until(
                EC.visibility_of_element_located((By.ID, 'assinaturas'))
            )
            
            dados_assinatura = {
                "Assinado por": driver.find_element(By.CSS_SELECTOR, 'strong.assinadoPor + span.branco').text,
                "CPF": driver.find_element(By.CSS_SELECTOR, 'strong.identificador + span.espaco').text,
                "Nº de série": driver.find_element(By.CSS_SELECTOR, 'strong.numserie + span.espaco').text,
                "Data da assinatura": driver.find_element(By.CSS_SELECTOR, 'strong.dataDaValidacao + span.espaco').text,
                "Status": driver.find_element(By.CSS_SELECTOR, 'strong.branco.frase').text
            }

            print("Resultado da validação:")
            for chave, valor in dados_assinatura.items():
                print(f"{chave}: {valor}")

            return dados_assinatura

    finally:
        driver.quit()

if __name__ == "__main__":
    
    result = validar_assinatura(r'C:\Users\rafin\OneDrive\Documentos\GREAT\seplag\sign_check\docs\test_03.jpeg')
    print(result)


Somente arquivos .pdf, .xml, .p7s e .json serão aceitos no campo de documento.
